# Economics Countries Master Dataset

**Purpose**: Create a consolidated country-year economic structure dataset for merging with conflict data.

**Output**: `economics-countries-master.csv`

**Coverage**: 
- 220 countries
- 1970-2023 (54 years)
- Sector percentages: Primary, Secondary, Tertiary
- Tourism % (2008-2023)
- GDP in USD

**Sources**:
1. World Bank GDP sectoral breakdown (sectoral composition)
2. UN Tourism SDG 8.9.1 (tourism %)
3. World Bank Development Indicators (GDP in USD)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully")

## Step 1: Load World Bank GDP Sectoral Data

Loading the GDP breakdown by sector. Header starts at row 3 (row index 2).

In [ ]:
# Load World Bank GDP sectoral data
df_gdp_sectoral = pd.read_excel(
    '../raw-data/World_Bank/Download-GDPcurrent-NCU-countries.xlsx',
    sheet_name=0,
    header=2
)

print(f"Loaded {len(df_gdp_sectoral):,} rows")
print(f"\nColumns: {list(df_gdp_sectoral.columns[:10])}...")
print(f"\nIndicators:")
for ind in df_gdp_sectoral['IndicatorName'].unique():
    count = len(df_gdp_sectoral[df_gdp_sectoral['IndicatorName'] == ind])
    print(f"  - {ind}: {count} countries")

df_gdp_sectoral.head()

## Step 2: Reshape from Wide to Long Format

Convert year columns (1970-2023) into rows for easier processing.

In [ ]:
# Get year columns (all numeric columns from 1970 onwards)
year_columns = [col for col in df_gdp_sectoral.columns if isinstance(col, (int, float)) and col >= 1970]

print(f"Year range: {min(year_columns)} to {max(year_columns)}")
print(f"Total years: {len(year_columns)}")

# Reshape to long format
df_long = df_gdp_sectoral.melt(
    id_vars=['CountryID', 'Country', 'Currency', 'IndicatorName'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

# Convert Year to integer
df_long['Year'] = df_long['Year'].astype(int)

print(f"\nReshaped to {len(df_long):,} rows")
df_long.head(10)

## Step 3: Pivot Indicators to Columns

Each indicator becomes a column for easier calculation.

In [ ]:
# Pivot so each indicator is a column
df_pivot = df_long.pivot_table(
    index=['CountryID', 'Country', 'Currency', 'Year'],
    columns='IndicatorName',
    values='Value',
    aggfunc='first'
).reset_index()

print(f"Pivoted to {len(df_pivot):,} rows (country-years)")
print(f"Columns: {len(df_pivot.columns)}")
print(f"\nCountries: {df_pivot['Country'].nunique()}")
print(f"Year range: {df_pivot['Year'].min()} to {df_pivot['Year'].max()}")

df_pivot.head()

## Step 4: Calculate Sector Percentages

**Sector Definitions**:
- **Primary**: Agriculture + Mining (extractive/agricultural)
- **Secondary**: Manufacturing + Construction (industrial)
- **Tertiary**: Trade/Retail + Transport + Other Services (services)

All calculated as % of Total Value Added.

In [ ]:
# Shorter column names for easier reference
df_pivot.columns = df_pivot.columns.str.strip()

# Map long indicator names to short names
col_map = {
    'Agriculture, hunting, forestry, fishing (ISIC A-B)': 'Agriculture',
    'Mining, Manufacturing, Utilities (ISIC C-E)': 'Mining_Manuf_Util',
    'Manufacturing (ISIC D)': 'Manufacturing',
    'Construction (ISIC F)': 'Construction',
    'Wholesale, retail trade, restaurants and hotels (ISIC G-H)': 'Trade_Retail',
    'Transport, storage and communication (ISIC I)': 'Transport',
    'Other Activities (ISIC J-P)': 'Other_Activities',
    'Total Value Added': 'Total_Value_Added',
    'Gross Domestic Product (GDP)': 'GDP'
}

df_pivot = df_pivot.rename(columns=col_map)

# Calculate Mining (Mining+Manuf+Util - Manufacturing)
df_pivot['Mining'] = df_pivot['Mining_Manuf_Util'] - df_pivot['Manufacturing']

# Calculate sector components
df_pivot['Primary_Value'] = df_pivot['Agriculture'] + df_pivot['Mining']
df_pivot['Secondary_Value'] = df_pivot['Manufacturing'] + df_pivot['Construction']
df_pivot['Tertiary_Value'] = df_pivot['Trade_Retail'] + df_pivot['Transport'] + df_pivot['Other_Activities']

# Calculate percentages (relative to Total Value Added)
df_pivot['Primary_%'] = (df_pivot['Primary_Value'] / df_pivot['Total_Value_Added']) * 100
df_pivot['Secondary_%'] = (df_pivot['Secondary_Value'] / df_pivot['Total_Value_Added']) * 100
df_pivot['Tertiary_%'] = (df_pivot['Tertiary_Value'] / df_pivot['Total_Value_Added']) * 100

# Validation: check that percentages sum to ~100%
df_pivot['Total_%'] = df_pivot['Primary_%'] + df_pivot['Secondary_%'] + df_pivot['Tertiary_%']

print("Sample calculations:")
print(df_pivot[['Country', 'Year', 'Primary_%', 'Secondary_%', 'Tertiary_%', 'Total_%']].head(10))

print("\nValidation - Total % distribution:")
print(df_pivot['Total_%'].describe())

## Step 5: Load and Merge UN Tourism Data

Tourism as % of GDP (2008-2023 only, 125 countries).

In [ ]:
# Load UN Tourism data
df_tourism = pd.read_excel(
    '../raw-data/UN_Tourism/UN_Tourism_8_9_1_TDGDP_04_2025.xlsx',
    sheet_name='SDG 8.9.1',
    header=0
)

print(f"Loaded {len(df_tourism):,} tourism records")
print(f"Countries: {df_tourism['GeoAreaName'].nunique()}")
print(f"Year range: {df_tourism['TimePeriod'].min()} to {df_tourism['TimePeriod'].max()}")

# Select relevant columns and rename
df_tourism_clean = df_tourism[['GeoAreaName', 'TimePeriod', 'Value']].copy()
df_tourism_clean.columns = ['Country', 'Year', 'Tourism_%']

print("\nSample tourism data:")
print(df_tourism_clean.head(10))

In [ ]:
# Merge tourism data with main dataset
df_master = df_pivot.merge(
    df_tourism_clean,
    on=['Country', 'Year'],
    how='left'
)

print(f"After merging tourism: {len(df_master):,} rows")
print(f"\nTourism data coverage:")
print(f"  - Records with tourism data: {df_master['Tourism_%'].notna().sum():,}")
print(f"  - Records without tourism data: {df_master['Tourism_%'].isna().sum():,}")

# Show countries with tourism data
countries_with_tourism = df_master[df_master['Tourism_%'].notna()]['Country'].nunique()
print(f"  - Countries with tourism data: {countries_with_tourism}")

## Step 6: Add GDP in USD

From World Bank Development Indicators for cross-country comparisons.

In [ ]:
# Load World Bank Development Indicators
df_dev_ind = pd.read_csv('../raw-data/World_Bank/world_bank_development_indicators.csv')

print(f"Loaded {len(df_dev_ind):,} development indicator records")
print(f"Countries: {df_dev_ind['country'].nunique()}")

# Extract year from date column
df_dev_ind['Year'] = pd.to_datetime(df_dev_ind['date']).dt.year

# Select GDP in USD column
df_gdp_usd = df_dev_ind[['country', 'Year', 'GDP_current_US']].copy()
df_gdp_usd.columns = ['Country', 'Year', 'GDP_USD']

print("\nSample GDP USD data:")
print(df_gdp_usd.head(10))

In [ ]:
# Merge GDP USD with master dataset
df_master = df_master.merge(
    df_gdp_usd,
    on=['Country', 'Year'],
    how='left'
)

print(f"After merging GDP USD: {len(df_master):,} rows")
print(f"\nGDP USD coverage:")
print(f"  - Records with GDP USD: {df_master['GDP_USD'].notna().sum():,}")
print(f"  - Records without GDP USD: {df_master['GDP_USD'].isna().sum():,}")

## Step 7: Create Final Master Dataset

Select and order columns for the final output.

In [ ]:
# Select final columns
df_final = df_master[[
    'Country',
    'Year',
    'Primary_%',
    'Secondary_%',
    'Tertiary_%',
    'Tourism_%',
    'GDP_USD'
]].copy()

# Round percentages to 2 decimal places
df_final['Primary_%'] = df_final['Primary_%'].round(2)
df_final['Secondary_%'] = df_final['Secondary_%'].round(2)
df_final['Tertiary_%'] = df_final['Tertiary_%'].round(2)
df_final['Tourism_%'] = df_final['Tourism_%'].round(2)

# Sort by Country and Year
df_final = df_final.sort_values(['Country', 'Year']).reset_index(drop=True)

print(f"Final dataset: {len(df_final):,} rows × {len(df_final.columns)} columns")
print(f"\nCountries: {df_final['Country'].nunique()}")
print(f"Year range: {df_final['Year'].min()} - {df_final['Year'].max()}")

df_final.head(20)

## Step 8: Data Validation

Quality checks before export.

In [ ]:
print("=== DATA QUALITY REPORT ===")
print("\n1. Missing Data Summary:")
print(df_final.isnull().sum())
print(f"\n   Missing data %:")
print((df_final.isnull().sum() / len(df_final) * 100).round(2))

print("\n2. Sector Percentage Validation:")
df_final['Total_%'] = df_final['Primary_%'] + df_final['Secondary_%'] + df_final['Tertiary_%']
print(df_final['Total_%'].describe())

# Check for rows that don't sum to ~100%
invalid_rows = df_final[(df_final['Total_%'] < 99) | (df_final['Total_%'] > 101)]
print(f"\n   Rows with total ≠ 100% (±1%): {len(invalid_rows):,}")

print("\n3. Sample Countries - Recent Years:")
for country in ['Afghanistan', 'Syria', 'United States', 'Germany', 'China']:
    sample = df_final[(df_final['Country'] == country) & (df_final['Year'] >= 2015)]
    if len(sample) > 0:
        print(f"\n{country}:")
        print(sample[['Year', 'Primary_%', 'Secondary_%', 'Tertiary_%', 'Tourism_%']])

print("\n4. Tourism Coverage by Year:")
tourism_by_year = df_final[df_final['Tourism_%'].notna()].groupby('Year').size()
print(tourism_by_year)

# Drop the temporary Total_% column
df_final = df_final.drop(columns=['Total_%'])

## Step 9: Export Master Dataset

Save to `processed-data/economics-countries-master.csv`

In [ ]:
# Export to CSV
output_path = '../processed-data/economics-countries-master.csv'
df_final.to_csv(output_path, index=False)

print(f"✓ Exported to: {output_path}")
print(f"\nDataset Summary:")
print(f"  - Total records: {len(df_final):,}")
print(f"  - Countries: {df_final['Country'].nunique()}")
print(f"  - Years: {df_final['Year'].min()} - {df_final['Year'].max()} ({df_final['Year'].nunique()} years)")
print(f"  - File size: {pd.read_csv(output_path).memory_usage(deep=True).sum() / 1024:.1f} KB")

print("\n✓ Economics Countries Master dataset created successfully!")
print("\nReady to merge with conflict data using Country + Year keys.")